# Path Enumeration for One Source-Sink Pair in a Directed Network

This notebook illustrates the use of the path algorithm on the directed graph *G*.

A path in graph *G* is a sequence of links such that the to node of each link is the from node of the next link. An elementary path is a path in which no node is visited more than once. A path between two nodes, *i* and *j*, in a graph is a path that starts at *i* and ends at *j*. The starting node is called the source node, and the ending node is called the sink node.

Below, we provide the steps to enumerate paths between a source node *D* and sink node *A* with an upper limit, 10, on cumulative path weight.

----------------
__Prepared by:__
Damian Herrick (<i class="fa fa-github" aria-hidden="true"></i>: [dtherrick](www.github.com/dtherrick))


## Imports

The modules below are needed for this exercise.

| Module             | Description                                                                        |
|:-------------------|:----------------------------------------------------------------------------------:|
| `os`               | Allows access to environment variables.                                            |
| `swat`             | SAS Python module that orchestrates communication with a CAS server.               |
| `pandas`           | Data management module we use for preparation of local data.                       |
| `graphviz.Digraph` | Used to display the organizational structure with the results of our reach query.  | 

In [1]:
import os
import swat
import pandas as pd
from pathlib import Path

from graphviz import Digraph

## Connect to our Viya (CAS) server

We now need to connect to our CAS server. Contact your SAS administrator for the proper credentials to connect to CAS using `python-swat`.

As a convention, I keep my CAS host and port set as environment variables. This allows me to both avoid placing user-specific data in notebooks, as well as adds a layer of security.

For ease-of-reading, I assign those environment variables into `host` and `port` variables, then pass them into the connection statement.

Once connected, we need access to the `PROC NETWORK` actions. In CAS terminology, that is an `actionset` so we load that now.

In [2]:
host = os.environ['CAS_HOST_ORGRD']
port = int(os.environ['CAS_PORT'])

conn = swat.CAS(host, port)

conn.loadactionset("network")

NOTE: Added action set 'network'.


[actionset]

 'network'

+ Elapsed: 0.0334s, user: 0.113s, sys: 0.035s, mem: 6.26mb

## Define the edgelist and weights locally as a pandas dataframe, then upload to CAS

In [3]:
colNames = ["from", "to", "weight"]

links = [
    ("A", "B", 1),
    ("A", "E", 1),
    ("B", "C", 1),
    ("C", "A", 6),
    ("C", "D", 1),
    ("D", "E", 3),
    ("D", "F", 1),
    ("E", "B", 1),
    ("E", "C", 4),
    ("F", "E", 1),
    ("E", "A", 1),
]

dfLinkSetIn = pd.DataFrame(links, columns=colNames)

conn.upload(dfLinkSetIn, casout=dict(name='LinkSetIn'))

NOTE: Cloud Analytic Services made the uploaded file available as table LINKSETIN in caslib CASUSERHDFS(daherr).
NOTE: The table LINKSETIN has been created in caslib CASUSERHDFS(daherr) from binary data uploaded to Cloud Analytic Services.


[caslib]

 'CASUSERHDFS(daherr)'

[tableName]

 'LINKSETIN'

[casTable]

 CASTable('LINKSETIN', caslib='CASUSERHDFS(daherr)')

+ Elapsed: 0.0397s, user: 0.05s, sys: 0.075s, mem: 167mb

## Calculate the paths from node *D* to *A* in our graph using the `network` actionset.

Since we've loaded our actionset, we can reference it using dot notation from our `connection` object.

Note that the Python code below is equivalent to this block of CASL:

```
proc network
   direction         = directed
   links             = mycas.LinkSetIn
   outNodes          = mycas.NodeSetOut;
   path
      source         = D
      sink           = A
      maxLinkWeight  = 10
      outPathsLinks  = mycas.PathLinks
      outPathsNodes  = mycas.PathNodes;
run;
```

In [4]:
conn.network.path(links         = {'name':'LinkSetIn'}, 
                  outnodes      = {'name':'NodeSetOut', 'replace':True}, 
                  direction     = 'directed', 
                  source        = 'D', 
                  sink          = 'A', 
                  maxlinkweight = 10, 
                  outpaths      = {'name':'PathLinks', 'replace':True}, 
                  outpathsnodes = {'name':'PathNodes', 'replace':True}
                 )

NOTE: The number of nodes in the input graph is 6.
NOTE: The number of links in the input graph is 11.
NOTE: Processing path enumeration using 80 threads across 5 machines.
NOTE: Processing path enumeration between 1 source nodes and 1 sink nodes.
NOTE: The algorithm found 3 paths.
NOTE: Processing path enumeration used 0.00 (cpu: 0.00) seconds.


[OutputCasTables]

                 casLib        Name Label  Rows  Columns                                           casTable
 0  CASUSERHDFS(daherr)  NodeSetOut           6        1  CASTable('NodeSetOut', caslib='CASUSERHDFS(dah...
 1  CASUSERHDFS(daherr)   PathLinks          10        7  CASTable('PathLinks', caslib='CASUSERHDFS(dahe...
 2  CASUSERHDFS(daherr)   PathNodes          13        5  CASTable('PathNodes', caslib='CASUSERHDFS(dahe...

[ProblemSummary]

 Problem Summary
 
             Name1           Label1       cValue1  nValue1
 0        numNodes  Number of Nodes             6      6.0
 1        numLinks  Number of Links            11     11.0
 2  graphDirection  Graph Direction      Directed      NaN

[SolutionSummary]

 Solution Summary
 
          Name1           Label1       cValue1   nValue1
 0  problemType     Problem Type          Path       NaN
 1       status  Solution Status            OK       NaN
 2     numPaths  Number of Paths             3  3.000000
 3      cpuTime         CPU Time          0.00  0.000000
 4     realTime        Real Time          0.00  0.001887

[numPaths]

 3

[solutionStatus]

 'OK'

[solutionStatusCode]

 0

+ Elapsed: 0.123s, user: 0.102s, sys: 0.14s, mem: 401mb

## Review the *PathLinks* table in CAS using the `fetch` action

* Verify that we identified three paths that exist between *D* and *A*.

In [5]:
conn.fetch('PathLinks')

,source,sink,path,order,from,to,weight
0,D,A,0.0,0.0,D,E,3.0
1,D,A,0.0,1.0,E,A,1.0
2,D,A,1.0,0.0,D,F,1.0
3,D,A,1.0,1.0,F,E,1.0
4,D,A,1.0,2.0,E,A,1.0
5,D,A,2.0,0.0,D,F,1.0
6,D,A,2.0,1.0,F,E,1.0
7,D,A,2.0,2.0,E,B,1.0
8,D,A,2.0,3.0,B,C,1.0
9,D,A,2.0,4.0,C,A,6.0


## Prepare data to easily visualize the three possible paths.

We need to conduct the following steps:
1. Make a local dataframe `dfPathLinks` that has only the nodes, path, and location ("order") from the CAS table `PathLinks`
2. Create a complete edgelist `dfComplete` that merges the edgelist we created above and the `dfPathLinks` dataframe we just created.
3. Use the complete edgelist to generate four plots - one of the source graph, and one for each path. We use Graphviz to generate these plots.

In [6]:
dfPathLinks = pd.DataFrame(conn.fetch(table=dict(name='PathLinks'))['Fetch'][['from', 'to', 'path', 'order']])

dfPathLinks

,from,to,path,order
0,D,E,0.0,0.0
1,E,A,0.0,1.0
2,D,F,1.0,0.0
3,F,E,1.0,1.0
4,E,A,1.0,2.0
5,D,F,2.0,0.0
6,F,E,2.0,1.0
7,E,B,2.0,2.0
8,B,C,2.0,3.0
9,C,A,2.0,4.0


In [7]:
dfComplete = pd.merge(dfLinkSetIn, dfPathLinks, how='left', left_on=['from', 'to'], right_on=['from', 'to'])

dfComplete

,from,to,weight,path,order
0,A,B,1,NaN,NaN
1,A,E,1,NaN,NaN
2,B,C,1,2.0,3.0
3,C,A,6,2.0,4.0
4,C,D,1,NaN,NaN
5,D,E,3,0.0,0.0
6,D,F,1,1.0,0.0
7,D,F,1,2.0,0.0
8,E,B,1,2.0,2.0
9,E,C,4,NaN,NaN


In [8]:
graph = dfLinkSetIn.to_dict(orient='records')
nodes = ['A', 'B', 'C', 'D', 'E', 'F']

list_paths = list(dfPathLinks['path'].unique())

# we have three paths but want four plots - the source network w/o paths, then a last one that it the source itself
# so we append a dummy path value (999) to the list of paths. when we see that we just generate the blank path
DUMMY_PATH = 999
list_paths.append(DUMMY_PATH)

for path in list_paths:
    path_suffix = 0 if path == DUMMY_PATH else int(path + 1)
    fname = Path(f"../dot/python_ex1_{path_suffix}").resolve()
    src_name = Path(f"../dot/python_ex1_{path_suffix}.dot").resolve()
    path_edges = dfComplete.loc[dfComplete['path'] == path][['from', 'to']].to_dict(orient='records')
    path_nodes = list(set([v for row in path_edges for k,v in row.items()]))
    path_tuples = [(row['from'], row['to']) for row in path_edges]

    dot = Digraph(format='png')

    for item in nodes:
        if item in path_nodes:
            dot.node(item, color='blue', penwidth='3')
        else:
            dot.node(item)

    for pair in graph:
        if (pair['from'], pair['to']) in path_tuples:
            dot.edge(pair['from'], pair['to'], label=str(pair['weight']), color='blue', penwidth='3')
        else:
            dot.edge(pair['from'], pair['to'], label=str(pair['weight']))

    dot.render(filename=fname.as_posix())
    fname.rename(src_name)

In [1]:
from IPython.display import HTML, Image

def _src_from_data(data):
    """Base64 encodes image bytes for inclusion in an HTML img element"""
    img_obj = Image(data=data)
    for bundle in img_obj._repr_mimebundle_():
        for mimetype, b64value in bundle.items():
            if mimetype.startswith('image/'):
                return f'data:{mimetype};base64,{b64value}'

def gallery(images, row_height='auto'):
    """Shows a set of images in a gallery that flexes with the width of the notebook.
    
    Parameters
    ----------
    images: list of str or bytes
        URLs or bytes of images to display

    row_height: str
        CSS height value to assign to all images. Set to 'auto' by default to show images
        with their native dimensions. Set to a value like '250px' to make all rows
        in the gallery equal height.
    """
    figures = []
    for image in images:
        if isinstance(image, bytes):
            src = _src_from_data(image)
            caption = ''
        else:
            src = image
            caption = f'<figcaption style="font-size: 0.6em">{image}</figcaption>'
        figures.append(f'''
            <figure style="margin: 5px !important;">
              <img src="{src}" style="height: {row_height}">
              {caption}
            </figure>
        ''')
    return HTML(data=f'''
        <div style="display: flex; flex-flow: row wrap; text-align: center;">
        {''.join(figures)}
        </div>
    ''')

In [4]:
from pathlib import Path

images = [image.as_posix() for image in Path('../dot/').glob('python_*.png')]

gallery(images)

## Clean up everything. 

* Make sure we know what tables we created, drop them, and close our connection.
(This is probably overkill, since everything in this session is ephemeral anyway, but good practice nonetheless).

In [9]:
table_list = conn.tableinfo()["TableInfo"]["Name"].to_list()

for table in table_list:
    conn.droptable(name=table, quiet=True)

conn.close()